In [ ]:
# Group : Groupe B2-4 [Equipe 8]
# AI name : PabloV2
# Members : Jules Peignier, Aymen Kallala 
# Description : PabloV2 est un greedy par zone de forte densité de fromage, il a deux états. 
#   -> Le premier où il va dans une zone de forte densité (selon la densité et la distance au joueur)
#   -> Le second où il se met en mode Greedy MR, le greedy amélioré

In [ ]:
MOVE_DOWN = 'D'
MOVE_LEFT = 'L'
MOVE_RIGHT = 'R'
MOVE_UP = 'U'
 
 
# Librarie containing FIFO structure

import numpy as np
import heapq

In [ ]:
def moves_from_locations (locations) :
    # Transform a series of locations into corresponding moves to realize it
  list_moves=[]
  for i in range(len(locations)-1) :
    list_moves.append(move_from_locations(locations[i],locations[i+1]))
  return list_moves

In [ ]:
#détermine un mouvement pour deux vecteurs voisins 
def move_from_locations (source_location, target_location) : 
    difference = (target_location[0] - source_location[0], target_location[1] - source_location[1])
    if difference == (0, -1) :
        return MOVE_DOWN
    elif difference == (0, 1) :
        return MOVE_UP
    elif difference == (1, 0) :
        return MOVE_RIGHT
    elif difference == (-1, 0) :
        return MOVE_LEFT


In [ ]:
# Use the routing table to find the sequence of locations from source to target
def find_route (routing_table, source_location, target_location) :
  path=[target_location] #On initialise une liste de points en commencant par la fin
  current_vertex=target_location
  while source_location!=current_vertex:
    #tant que le point de départ n'est pas désigné comme destination 
    parent=routing_table[current_vertex]
    #on ajoute les parents à l'itinéraire
    path.append(parent)
    # le parent devient le vertex de destination
    current_vertex=parent
  reversed_path = []
 
  while path != [] :
      reversed_path.append(path.pop())
  return reversed_path
  #on retourne la liste pour avoir l'itinéraire du départ vers l'arrivée

In [ ]:
def meta_graph_route_to_route (meta_graph_route, routing_tables) :
  final_path=[]
  for i in range (len(meta_graph_route)-1):
    final_path+=find_route (routing_tables[meta_graph_route[i]], meta_graph_route[i], meta_graph_route[i+1])[:-1]
  final_path.append(meta_graph_route[-1])
  return final_path
    # Return the sequence of locations in the maze to perform a route in the meta-graph

In [ ]:
def push_to_structure (element:tuple,priority_queue) :
  heapq.heappush(priority_queue,element)

def pop_from_structure (priority_queue) :
   return heapq.heappop(priority_queue)

# Implementation of BFS in order to create a routing table 
    
def traversal (start_vertex:tuple, graph:dict) :

    priority_queue = []
    distance={}
    #initialitation à l'infini des distances
    for elt in graph:
      distance[elt]=10**50

    # Add the starting vertex with None as parent
    push_to_structure((0,start_vertex),priority_queue) 
 
    # Initialize the outputs
    current_vertex=start_vertex
    explored_vertices = [] 
    routing_table = {} 

    distance[current_vertex]=0
    routing_table[current_vertex]=None
    
    # Iterate while some vertices remain
    while priority_queue!=[] :
        
        current_distance,current_vertex=pop_from_structure(priority_queue)
        
        explored_vertices.append(current_vertex)
            
        for neighbor in graph[current_vertex]:
          if neighbor not in explored_vertices:
           
            push_to_structure((current_distance+graph[current_vertex][neighbor],neighbor),priority_queue)
          
          if current_distance+graph[current_vertex][neighbor] < distance[neighbor] :
           
            distance[neighbor] = current_distance+graph[current_vertex][neighbor]
           
            routing_table[neighbor] = current_vertex
                    
    return routing_table,distance
 
# Algorithm providing the shortest path between starting location and the piece of cheese

In [ ]:
def build_meta_graph (maze_map, locations) :
  routing_table_cheeses={}
  meta_graph={ fromage : {} for fromage in locations}
  for start_fromage in locations : 
    routing_table, distance = traversal(start_fromage , maze_map )
    routing_table_cheeses[start_fromage] = routing_table
    for target_fromage in locations :
      meta_graph[start_fromage][target_fromage] = distance[target_fromage]
  return meta_graph, routing_table_cheeses
#return meta_graph,routing_table_cheeses
    # Return the meta-graph and all necessary routing tables

In [ ]:
def greedyTurn(current_vertex, meta_graph, pieces_of_cheese):
  best_length=10**5
  best_fromage=current_vertex
  for fromages in meta_graph :
    if fromages in pieces_of_cheese:
      if meta_graph[current_vertex][fromages] < best_length :
        best_length =  meta_graph[current_vertex][fromages]
        best_fromage = fromages
  return best_fromage

In [ ]:
def tri(fromages,meta_graph,player_location):
    
    densite={}
    best_alpha1,best_alpha2=0,0
    best_target1, best_target2=player_location, player_location

    for target in fromages:
        densite[target]=0
        if 20 < meta_graph[player_location][target] < 300:
          for fromage in fromages:
              if meta_graph[target][fromage] < 20:
                  
                  densite[target]+=1
          if densite[target] > 5:
            d=densite[target]
            d=d*np.log(d)
            l=meta_graph[player_location][target]
            l=l**0.5
            alpha=d/l

            if alpha > best_alpha1 : 
                best_alpha1=alpha
                best_target1=target
            elif alpha > best_alpha2 :
                best_alpha2=alpha
                best_target2=target
    return best_target1, best_target2

In [ ]:
def maze_reducer(maze_map,player_location,pieces_of_cheese):
  maze_reduced=maze_map
  l=[]
  for i in range (12):
    for case in maze_reduced:
      if len(maze_reduced[case])==1 and case not in [player_location]+pieces_of_cheese :
        l.append(case)
        for voisin in maze_reduced[case]:
          del maze_reduced[voisin][case]
    while l != [] :
      del maze_reduced[l[0]]
      l.pop(0)
  return maze_reduced

In [ ]:
def delete(listes,elt):
  liste=listes[:]
  i=0
  while i < len(liste):
    if liste[i] != elt :
        i+=1
    else:
        liste.pop(i)
        return liste
  return liste

In [ ]:
def priority(current_vertex,target,meta_graph,pieces_of_cheese):
  #initialisation (racccourcis pour la suite)
  gt=greedyTurn(current_vertex, meta_graph, pieces_of_cheese)
  mg=meta_graph
  cr=current_vertex
  t=target

  if gt==t:
    return [True,gt]
  elif abs(mg[cr][t]-(mg[cr][gt]+mg[gt][t])) < 10 :
    return [True,gt]
  return [False,gt]
  

In [ ]:

def preprocessing (maze_map, maze_width, maze_height, player_location, opponent_location, pieces_of_cheese, time_allowed) :
       # Implement the list of moves to perform in the preprocess
    
    global maze_reduced
    global meta_graph
    global routing_table_cheeses
    
    global target
    global best_target
    global best_target1
    global best_target2

    global transition
    transition=0

    target=player_location
    best_target=player_location


    maze_reduced=maze_reducer(maze_map,player_location,pieces_of_cheese)
    meta_graph, routing_table_cheeses= build_meta_graph (maze_reduced, [player_location]+pieces_of_cheese)
    best_target1, best_target2 = tri(pieces_of_cheese,meta_graph,player_location)

    

In [ ]:
#turn va renvoyer le dernier move de la liste "list_moves" à chaque appel
dernier_fromage=(0,0)
list_moves=[]
def turn (maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed) :
  
    global best_target1
    global best_target2

    if best_target1 in pieces_of_cheese or best_target2 in pieces_of_cheese:
      return Area_operating_mode(maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed)
    return GreedyTurn_operating_mode(maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed)
      

In [ ]:
def Area_operating_mode(maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed):
    
    global maze_reduced
    global meta_grah
    global routing_table_cheeses
    
    global list_moves
    
    global target 
    global best_target
    global best_target1
    global best_target2
    global dernier_fromage
    

    if best_target1 in pieces_of_cheese:
      best_target=best_target1

    else:

      best_target=best_target2
    
    target=best_target
    dernier_fromage=target

    if list_moves==[]:

      pocModified=pieces_of_cheese[:]
      path=[]
      current_vertex=player_location
      prio=priority(current_vertex,best_target,meta_graph,pocModified)

      while current_vertex != best_target:

        if prio[0] == True :

          current_vertex=prio[1]
          path.append(current_vertex)
          pocModified=delete(pocModified,current_vertex)
          prio=priority(current_vertex,best_target,meta_graph,pocModified)
        
        else :

     
       
          current_vertex=prio[1]
          pocModified=delete(pocModified,current_vertex)
          prio=priority(current_vertex,best_target,meta_graph,pocModified)
      
      list_moves=moves_from_locations(meta_graph_route_to_route ([player_location]+path+[best_target], routing_table_cheeses))
      return list_moves.pop(0) 
    return list_moves.pop(0)

In [ ]:
def GreedyTurn_operating_mode(maze_map, maze_width, maze_height, player_location, opponent_location, player_score, opponent_score, pieces_of_cheese, time_allowed):    
    
    global maze_reduced
    global meta_grah
    global routing_table_cheeses
    
    global list_moves
    
    global target 
    global best_target
    global dernier_fromage

    global transition
    
    if transition == 0 and target!=player_location :

      target=greedyTurn(dernier_fromage, meta_graph, pieces_of_cheese)
      particular_routing_table=traversal(player_location , maze_reduced )[0]
      list_moves=moves_from_locations(find_route (particular_routing_table, player_location, target))
      transition = 1
      return list_moves.pop(0)
      
    
    if target==player_location:

      dernier_fromage=target
      target=greedyTurn(player_location, meta_graph, pieces_of_cheese)
      list_moves=moves_from_locations(meta_graph_route_to_route ([player_location,target], routing_table_cheeses))
      return list_moves.pop(0)
    
    elif target in pieces_of_cheese:

      return list_moves.pop(0)

    else :

      target=greedyTurn(dernier_fromage, meta_graph, pieces_of_cheese)
      particular_routing_table=traversal(player_location , maze_reduced )[0]
      list_moves=moves_from_locations(find_route (particular_routing_table, player_location, target))
      return list_moves.pop(0)